In [ ]:
import requests
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from pysheds.grid import Grid
import pysheds
import numpy as np
import rasterio as rio
from rasterio.merge import merge


In [ ]:
import ee
import geemap
ee.Authenticate()  # Authenticate if not already done
# Initialize GEE
ee.Initialize(project="ee-tut-452819")

In [ ]:
houston = gpd.read_file("data_TX\HCAD_Harris_County_Boundary.geojson")
aoi = ee.FeatureCollection("projects/ee-tut-452819/assets/HCAD_Harris_County_Boundary").geometry()

In [ ]:
def get_tile_name(lat, lon):
    
    lat_tile = f"n{int(abs(lat)):02d}" if lat >= 0 else f"s{int(abs(lat)):02d}"
    lon_tile = f"w{int(abs(lon)):03d}" if lon < 0 else f"e{int(abs(lon)):03d}"
    return f"{lat_tile}{lon_tile}"

In [ ]:
def get_tiles_in_bbox(min_lat, max_lat, min_lon, max_lon):
    
    tile_list = []
    for lat in range(int(min_lat), int(max_lat) + 1):
        for lon in range(int(min_lon), int(max_lon) + 1):
            tile_list.append(get_tile_name(lat, lon))
    return tile_list

tiles = get_tiles_in_bbox(30, 31, -96.12, -95)
print("Tiles to Download:", tiles)

In [ ]:
def download_dem_1arc(tile_name, save_path="."):
    
    base_url = "https://prd-tnm.s3.amazonaws.com/StagedProducts/Elevation/1/TIFF/current"
    url = f"{base_url}/{tile_name}/USGS_1_{tile_name}.tif"
    
    output_file = os.path.join(save_path, f"{tile_name}.tif")
    
    response = requests.get(url, stream=True)
    if response.status_code == 200:
        with open(output_file, "wb") as file:
            for chunk in response.iter_content(1024):
                file.write(chunk)
        print(f"✅ DEM Tile {tile_name} downloaded successfully!")
    else:
        print(f"❌ Error: Tile {tile_name} not found. Check availability.")
        print(url)

save_directory = "DEM_Tiles_TX"
os.makedirs(save_directory, exist_ok=True)  # Create folder if it doesn't exist

for tile in tiles:
    if f"{tile}.tif" not in os.listdir(save_directory):
        print(f"Downloading {tile}...")
        download_dem_1arc(tile, save_directory)
    else:
        print(f"Tile {tile} already exists. Skipping download.")

In [ ]:
inputfolder= 'DEM_Tiles_TX'
outfolder = 'data_TX/inundation_wo_thresh'


threshold = 1.5

for file in os.listdir(inputfolder):  
    print(file)  
    if file.endswith('.tif'):
        
        tiffile = os.path.join(inputfolder, file)
        print(tiffile)
        try:
          
          grid = Grid.from_raster(tiffile)
          
          dem = grid.read_raster(tiffile)

          pit_filled_dem = grid.fill_pits(dem)

          flooded_dem = grid.fill_depressions(pit_filled_dem)

          inflated_dem = grid.resolve_flats(flooded_dem)

          fdir = grid.flowdir(inflated_dem)

          acc = grid.accumulation(fdir)

          hand = grid.compute_hand(fdir, dem, acc > 200) # tells HAND  to use the accumulation grid to determine which pixels are in the channel network (acc > 200)
          # so only consider flows the 200+ upstream pixels as channel network (ignores small channels and noise)
          
          #inundation_extent = np.where(hand < threshold, threshold - hand, np.nan)
          #inundation_extent = np.where(hand < threshold, 1, 0)
          inundation_extent= hand
          with rio.open(tiffile) as original_file:
            outpath = os.path.join(outfolder, f"{os.path.basename(tiffile)}")
            print(outpath)
            with rio.open(outpath, "w", **original_file.meta) as dest:
              dest.write(inundation_extent, indexes = 1)
              
        except:
            continue

In [ ]:
with rio.open(r'data_TX\inundation_wo_thresh\n30w095.tif') as tif:


  array = tif.read()

  plt.imshow(array[0], cmap='Blues', vmin=0, vmax=10)
  plt.title('Inundation Depth Map (1 arc-second DEM)')
  print(tif.profile)
  print(tif.bounds)
  print(array[0].max())

In [ ]:
inputfolder= 'data_TX/inundation_wo_thresh'
outfile = 'data_TX/mosacied_wo_thresh.tif'

tiflist = []

for file in os.listdir(inputfolder):    
    if file.endswith('.tif'):
        tiffile = os.path.join(inputfolder, file)
        tiflist.append(tiffile)


src_files_to_mosaic = []
for fp in tiflist:
    try:
        tile_src = rio.open(fp)
        tile_bounds = tile_src.bounds
        
        src_files_to_mosaic.append(tile_src)
    except:
        continue

print('The number of mosaiced tiles is:', len(src_files_to_mosaic))

mosaic, out_trans = merge(src_files_to_mosaic)
print('You have mosaiced the results')

out_meta = tile_src.meta.copy()
out_meta.update({"driver": "GTiff",
                  "height": mosaic.shape[1],
                  "width": mosaic.shape[2],
                  "transform": out_trans, 
                  "compress": 'lzw',
                  'BIGTIFF': 'YES'
                  }
               )

with rio.open(outfile, "w", **out_meta) as dest:
     dest.write(mosaic)

In [ ]:
with rio.open('data_TX/mosacied_wo_thresh.tif') as mosaic:


  mosaic_array = mosaic.read()
  
  plt.imshow(mosaic_array[0], cmap='Blues', vmin=0, vmax=5)
  plt.title('Inundation Depth Map')

In [ ]:
with rio.open('data_TX/mosacied_wo_thresh.tif') as mosaic:
    mosaic_array = mosaic.read()
    extent = [mosaic.bounds.left, mosaic.bounds.right, mosaic.bounds.bottom, mosaic.bounds.top]

# Plot the raster
fig, ax = plt.subplots(figsize=(10, 10))
ax.imshow(mosaic_array[0], cmap='Blues', vmin=0, vmax=5, extent=extent)
ax.set_title('Inundation Depth Map')

# Plot the Houston GeoDataFrame
houston.plot(ax=ax, facecolor='none', edgecolor='red', linewidth=2)

plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)
im = ax.imshow(acc, zorder=2,
               cmap='cubehelix',
               norm=colors.LogNorm(1, acc.max()),
               interpolation='bilinear')
plt.colorbar(im, ax=ax, label='Upstream Cells')
plt.title('Flow Accumulation', size=14)
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)
im = ax.imshow(hand, zorder=2,
               cmap='cubehelix',
               norm=colors.LogNorm(1, acc.max()),
               interpolation='bilinear')
plt.colorbar(im, ax=ax, label='Upstream Cells')
plt.title('Hand', size=14)
plt.tight_layout()
print(acc.max())

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
fig.patch.set_alpha(0)
dem_view = grid.view(dem, nodata=np.nan)
plt.imshow(dem_view, extent=grid.extent, cmap='Greys', zorder=1)
plt.imshow(inundation_extent, extent=grid.extent,
           cmap='Blues', vmin=-5, vmax=10, zorder=2)
plt.grid(zorder=0)
plt.title('Inundation depths (constant channel depth)', size=14)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.tight_layout()